In [1]:
import numpy as np
import scipy.ndimage, scipy.optimize, scipy.io
import os, sys, time, warnings
import matplotlib.pyplot as plt
%matplotlib inline
import tifffile
import h5py
#sys.path.insert(0, '/Volumes/Development/Spim_GUI/j_postacquisition')
from tqdm import tqdm_notebook as tqdm

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 25 days


In [2]:
matPath = '../SIsoftware/PSFmatrix/PSFmatrix_M22.2NA0.5MLPitch125fml3125from-110to110zspacing4Nnum19lambda520n1.33.mat'

warnings.warn('WARNING: Switched to faster matrix for testing')
matPath = '/Users/jonny/Movies/PSFmatrix/PSFmatrix_M40NA0.95MLPitch150fml3000from-26to0zspacing2Nnum15lambda520n1.0.mat'

/Users/jonny/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: WARNING: Switched to faster matrix for testing
  This is separate from the ipykernel package so we can avoid doing imports until


In [69]:
# Load the matrices from the .mat file.
# This is slow since they must be decompressed and are rather large! (9.5GB each, in single-precision FP)
print('Opening .mat file')
sys.stdout.flush()
with h5py.File(matPath, 'r') as f:
    print('Loading H')
    sys.stdout.flush()
    H = f['H'].value
    print('Loading Ht')
    sys.stdout.flush()
    Ht = f['Ht'].value
    print('Loading misc')
    sys.stdout.flush()
    CAindex = f['CAindex'].value

print(H.dtype, H.shape, Ht.shape, CAindex.shape)
print(CAindex.transpose())

Load H
Load Ht
Load misc
float32 (14, 15, 15, 391, 391) (14, 15, 15, 391, 391) (2, 14)
[[   1.   16.   31.   46.   61.   76.   91.  106.  121.  136.  151.  166.
   166.  166.]
 [ 391.  376.  361.  346.  331.  316.  301.  286.  271.  256.  241.  226.
   226.  226.]]


## Objects stored in the .mat file

### Optical parameters from GUI: [? means I am not sure if or where it is stored]

M<br>
NA<br>
d    "fml" in GUI (stored here in units of m)<br>
pixelPitch is "ML pitch" / "Nnum" (stored here in units of m)<br>
? n<br>
? wavelength<br>

### User parameters from GUI:

OSR<br>
zspacing<br>
? z-min<br>
? z-max<br>
Nnum<br>


### Misc parameter:

fobj (can presumably be deduced from mag, NA etc?)<br>

### The actual arrays:

H:             shape (56, 19, 19, 343, 343), type "f4"<br>
Ht:            shape (56, 19, 19, 343, 343), type "f4"<br>

### Information about object space:

x1objspace:    x pixel positions in object space (19 elements across one lenslet)<br>
x2objspace:    y pixel positions in object space (19 elements across one lenslet)<br>
x3objspace:    z pixel positions in object space (56 z planes)<br>
x1space:       x pixel positions in lenslet space (19 elements across one lenslet)<br>
x2space:       y pixel positions in lenslet space (19 elements across one lenslet)<br>

### Not sure what these are exactly:

CAindex:       shape (2, 56) - something about the start and end index of the PSF array, for each z plane.<br>
CP:            shape (343, 1)<br>
MLARRAY:       shape (1141, 1141), type "|V16"<br>
objspace:      shape (56, 1, 1)<br>
settingPSF:    You would think this contains the GUI parameters, but e.g. print(f['settingPSF']['M'].value) gives a strange 3x1 array [50, 50, 46, 50] etc...?<br>


In [4]:
# This function copy-pasted from:
# https://stackoverflow.com/questions/3731093/is-there-a-python-equivalent-of-matlabs-conv2-function
# The intention is to exactly replicate the conv2 function in Matlab.
# The issue is that scipy and matlab handle edge effects slightly differently

from scipy.ndimage.filters import convolve
from scipy.signal import convolve2d, fftconvolve


def conv2(x,y,mode='same'):
    # I took inspiration from the code at:
    # https://stackoverflow.com/questions/3731093/is-there-a-python-equivalent-of-matlabs-conv2-function
    # but I have attempted to achieve the same thing by slight zero-padding instead of by changing the origin.
    # That enables me to use fftconvolve instead of convolve (which makes the code much, much faster!)
    #print('conv', x.shape, y.shape)
    if not(mode == 'same'):
        raise Exception("Mode not supported")

    # Add singleton dimensions
    if (len(x.shape) < len(y.shape)):
        dim = x.shape
        for i in range(len(x.shape),len(y.shape)):
            dim = (1,) + dim
        x = x.reshape(dim)
    elif (len(y.shape) < len(x.shape)):
        dim = y.shape
        for i in range(len(y.shape),len(x.shape)):
            dim = (1,) + dim
        y = y.reshape(dim)

    # Apply padding to reproduce the exact same results as Matlab
    padShape = np.zeros((len(x.shape), 2), dtype='int32')
    for i in range(len(x.shape)):
        if ((x.shape[i] - y.shape[i]) % 2 == 0) and (x.shape[i] > 1) and (y.shape[i] > 1):
            padShape[i,0] = 1
        else:
            padShape[i,1] = 1
            #print('Pad axis', i)
    xp = np.pad(x, padShape, 'constant')
    
    if False:
        # Original (much slower) code for comparison
        origin = ()
        for i in range(len(x.shape)):
            if ((x.shape[i] - y.shape[i]) % 2 == 0 and
                 x.shape[i] > 1 and
                 y.shape[i] > 1):
                origin = origin + (-1,)
            else:
                origin = origin + (0,)

            t1 = time.time()
            z = convolve(x,y, mode='constant', origin=origin)
            t2 = time.time()

    zp = fftconvolve(xp, y, mode='same')
        
    # Apply padding to reproduce the exact same results as Matlab
    # I have just empirically found that this works.
    assert(len(zp.shape) == 2)   # I have only implemented this, because I can't find a generic function for N axes
    unpadShape = np.zeros((len(x.shape), 2), dtype='int32')
    for i in range(len(x.shape)):
        if (y.shape[i] % 2 == 0):#    TODO: check if it matters that they are >1...    and (x.shape[i] > 1) and (y.shape[i] > 1):
            unpadShape[i,0] = 1
            unpadShape[i,1] = x.shape[i]+1
        else:
            unpadShape[i,0] = 0
            unpadShape[i,1] = x.shape[i]
    t3 = time.time()
    zup = zp[unpadShape[0,0]:unpadShape[0,1], unpadShape[1,0]:unpadShape[1,1]]
    t4 = time.time()

    return zup

In [5]:
# Note: I am a little unsure how to interpret the arrays I have loaded from the .mat.
# From looking at how H and CAindex are accessed, it looks as if the shapes I have loaded
# are the reversal of the shape ordering as expected in Matlab.
# I suppose that makes sense given that matlab is column-major in its array accesses.
# The data has been loaded from disk in the order it is *stored*,
# and I therefore need to flip around all the matlab array index ordering 
# (e.g. matlabArray(1,2,3) becomes pythonArray[3,2,1])

In [66]:
# Note: H.shape in python is (56, 19, 19, 343, 343)

def forwardProjectACC(H, realspace, CAindex):
    Nnum = H.shape[2]
    zerospace = np.zeros((realspace.shape[1], realspace.shape[2]), dtype='float32')
    TOTALprojection = zerospace;

    # Iterate over each lenslet pixel
    for aa in tqdm(range(Nnum), desc='Forward-project - x'):
        for bb in tqdm(range(Nnum), leave=False, desc='Forward-project - y'):
            # Iterate over each z plane
            for cc in range(realspace.shape[0]):
                # I think this extracts the part of H that is nonzero for this z plane (and lenslet pixel)
                Hs = H[cc, bb, aa, CAindex[0,cc]-1:CAindex[1,cc], CAindex[0,cc]-1:CAindex[1,cc]]#.squeeze()
                print(aa, bb, cc, H.shape, Hs.shape)
                # Create a workspace representing just the voxels cc,bb,aa behind each lenslet (the rest is 0)
                tempspace = zerospace.copy()
                tempspace[bb::Nnum, aa::Nnum] = realspace[cc, bb::Nnum, aa::Nnum]
                # Compute how those voxels project onto the sensor
                projection = conv2(tempspace, Hs, 'same')
                # Accumuate that signal onto the sensor
                TOTALprojection = TOTALprojection + projection
    return TOTALprojection


def ConvolveManual(projection, aa, bb, Hts):
    tempSlice = np.zeros(projection.shape, dtype='float32')
    rangeA = range(aa, projection.shape[0], Nnum)
    rangeB = range(bb, projection.shape[1], Nnum)
    assert((Hts.shape[0] % 2) == 1)
    assert((Hts.shape[0] == Hts.shape[1]))
    centH = int(Hts.shape[0]/2)
    for a in rangeA:
        x0 = np.minimum(a, centH)
        x1 = np.minimum(projection.shape[0]-a, centH+1)
        for b in rangeB:
            y0 = np.minimum(b, centH)
            y1 = np.minimum(projection.shape[1]-b, centH+1)
            tempSlice[a-x0:a+x1, b-y0:b+y1] += Hts[centH-x0:centH+x1, centH-y0:centH+y1] * projection[a, b]
    return tempSlice
# Other thoughts for speeding things up: we are repeatedly convolving with the same submatrix of Ht.
# We could precalculate its Fourier transform. I don't think that would speed things up much, though,
# because that is small compared to the size of 'projection'.
   
    
    
    
def backwardProjectACC(Ht, projection, CAindex):
    print(projection.shape)
    x3length = Ht.shape[0]
    Nnum = Ht.shape[2]
    print((x3length, projection.shape[0], projection.shape[1]))
    Backprojection = np.zeros((x3length, projection.shape[0], projection.shape[1]), dtype='float32')
    zeroSlice = np.zeros(projection.shape, dtype='float32')  # TODO: I am assuming projection is a 2D array here

    # Iterate over each z plane
    for cc in tqdm(range(x3length), desc='Back-project - z'):
        sys.stdout.flush()
        tempSliceBack = zeroSlice.copy()
        print('Ht is %dx%d' % (CAindex[1,cc]-CAindex[0,cc], CAindex[1,cc]-CAindex[0,cc]))
        # Iterate over each lenslet pixel
        for aa in tqdm(range(1), leave=False, desc='y'):
            for bb in tqdm(range(Nnum), leave=False, desc='x'):    
                Hts = Ht[cc, bb, aa, CAindex[0,cc]-1:CAindex[1,cc], CAindex[0,cc]-1:CAindex[1,cc]]#.squeeze()             
                tempSlice = zeroSlice.copy()
                if False:
                    # I tried doing this manually, given how sparse the array is,
                    # but it wasn't actually any faster.
                    tempSliceBack = tempSliceBack + ConvolveManual(projection, aa, bb, Hts)
                else:
                    tempSlice[aa::Nnum, bb::Nnum] = projection[aa::Nnum, bb::Nnum]
                    tempSliceBack = tempSliceBack + conv2(tempSlice, Hts, 'same')
        Backprojection[cc,:,:] = Backprojection[cc,:,:] + tempSliceBack
    return Backprojection

In [61]:
Nnum = 19
projection = np.zeros((19*110,19*110))
aa = 0
bb = 0
projection[aa,bb] = 1
projection[19+aa,19+bb] = 2
Hts = np.ones((303,303))#np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

t1 = time.time()
result1 = ConvolveManual(projection, aa, bb, Hts)

t2 = time.time()

tempSlice = np.zeros(projection.shape, dtype='float32')
tempSlice[aa::Nnum, bb::Nnum] = projection[aa::Nnum, bb::Nnum]
result2 = conv2(tempSlice, Hts, 'same')
t3 = time.time()
print('manual', t2-t1)
print('scipy ', t3-t2)

difference = result1 - result2
print(np.max(np.abs(difference)))

manual 6.741102933883667
scipy  1.4916081428527832
2.14381620499e-14


In [62]:
def deconvRL(Htf, maxIter, Xguess):
    for i in range(maxIter):
        t0 = time.time()
        HXguess = forwardProjectACC(H, Xguess, CAindex)
        HXguessBack = backwardProjectACC(Ht, HXguess, CAindex)
        errorBack = Htf / HXguessBack
        Xguess = Xguess * errorBack
        Xguess[np.where(np.isnan(Xguess))] = 0
        ttime = time.time() - t0
        print('iter %d | %d, took %.1f secs' % (i+1, maxIter, ttime))
    return Xguess

In [63]:
# Load the input image
LFmovie = tifffile.imread('/Volumes/Development/SIsoftware/Data/01_Raw/exampleData/20131219WORM2_small_full_neg_X1.tif')
LFmovie = LFmovie[np.newaxis,:,:]

In [64]:
maxIter = 8

In [67]:
for frame in range(1):
    LFIMG = LFmovie[frame].astype('float32')
    t0 = time.time()
    Htf = backwardProjectACC(Ht, LFIMG, CAindex)
    print('iter 0 | %d, took %.1f secs' % (maxIter, time.time()-t0))
    Xguess = Htf;    # Or do it differently if !indepIter
    
    #Xguess = deconvRL(Htf, maxIter, Xguess)
    
    
# Matlab (with multithreading) takes:
#  iter 0 | 8, took 192.6506 secs
#  iter 1 | 8, took 516.5403 secs


(2160, 2560)
(14, 2160, 2560)


Ht is 390x390


/Users/jonny/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:64: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Ht is 360x360


KeyboardInterrupt: 

In [ ]:
if False:
    aa = np.array([[1, 2, 3, 4, 5], [4, 5, 6, 7, 8], [7, 8, 9, 10, 11]]).astype('float32')
    bb = np.array([[1, 2, 3, 4], [4, 5, 6, 7], [7, 8, 9, 10]]).astype('float32')

    aa = np.array([[1, 2, 3, 4, 5]*100, [4, 5, 6, 7, 8]*100, [7, 8, 9, 10, 11]*100]).astype('float32')
    #aa = np.zeros((100, 100)).astype('float32')
    bb = aa.copy()
    #conv2(aa, bb)
    #conv2(np.pad(aa, ((1,0),(0,0)), 'constant'), bb)

if False:
    r = range(110,112)
    for a1 in r:
        for a2 in r:
            for b1 in r:
                for b2 in r:
                    aa = np.random.random((a1,a2))
                    bb = np.random.random((b1,b2))
                    #print(a1, a2, b1, b2)
                    conv2(aa,bb)